# Easy Problem

We want to minimize the total cost of deliveries!

In [2]:
import numpy as np
import matplotlib as plt

## Parameters

We have the following problem parameters:
- Cost of the DC in euros $C = 25000$
- Maximum number of DCs $N = 20$
- Expected deliveries per 1000 people $D = 0.2$
- Cost per kilometer of each delivery $K = 1$
- Radius of the Earth $R = 6371.009$

In [3]:
PARAM = {
    "C": 25000,
    "N": 20,
    "D": 0.2,
    "K": 1,
    "R": 6371.009
}

# Model

We can then construct the model, where the decision variables are binary and correspond to the answer to the question "Will there be a distribution center in this town?".

Let the towns be represented by a number $i$, then $x_i$ is $1$ if there is a DC in $i$ and $0$ otherwise.

We can then model the problem as follows:
$$
\begin{alignat*}{2}

\min_{x_i} &\quad& &K \times \text{Sum of the distance from each city to the closest DC} - C \times \sum_i x_i
\\\\

\text{s.t}& &&\sum x_i \in \{0, 1\}
\\
&&& \sum_i x_i \in [1, 20]

\end{alignat*}
$$

## Data

Open the data we'll work with.

In [31]:
data = np.loadtxt("PopulationCountPT-2020.csv", delimiter=",", dtype=object)

index, name, pop, lat, lon = (data[1:, i] for i in range(5))

pop = pop.astype("float64")
lat = lat.astype("float64")
lon = lon.astype("float64")

# Calculate expected number of deliveries
val = pop * PARAM["D"]


# Convert the lats and lons to radians
lat = lat * np.pi / 180
lon = lon * np.pi / 180


# Number of cities
NC = int(index[-1])
print(NC)

379


## Organize the Data

We'll organize the data that relates to each city into a dict and store the distance between any pair of cities in a matrix.

In [43]:
def create_city(id: int, name: str, val: int, lat: float, lon: float) -> dict:
    return {
        "id": id,
        "name": name,
        "val": val,
        "lat": lat,
        "lon": lon
    }


def dist(city1: dict, city2: dict) -> float:
    global PARAM
    R = PARAM["R"]

    theta = min(city1["lat"], city2["lat"])

    dtheta = abs(city1["lat"] - city2["lat"])
    dphi = abs(city1["lon"] - city2["lon"])

    return R * np.sin(theta) *dphi + R * dtheta
    


# List with all the cities
CC = []
for id, n, v, la, lo in zip(index, name, val, lat, lon):
    CC += [create_city(int(id)-1, n, v, la, lo)]


DD = np.zeros((NC, NC))
for i1, city1 in enumerate(CC):
    for i2, city2 in enumerate(CC[i1:]):
        DD[i1, i1 + i2] = dist(city1, city2)

# print(DD[:5, :5])

[[  0.         306.85927257  10.91193996 364.69597021  38.35659918]
 [  0.           0.         309.54427733  58.52795115 311.10935063]
 [  0.           0.           0.         367.39171335  49.2401145 ]
 [  0.           0.           0.           0.         368.89038262]
 [  0.           0.           0.           0.           0.        ]]


# Solving the Problem

We are going to use the following "greedy" heuristic for solving the problem:
- Calculate the best place to put each DC and put it there!

To implement it, we are going to do the following:
- Compute the distances between every pair of cities and order them from closest to farthest.
- Calculate the value that we get from putting a DC in each available city, supposing each city is supplied by the closest DC to it, and choose the highest.

## Preliminaries